## R-CNN, Fast R-CNN
- Region Proposal 추출, 각 region proposal 별로 CNN 연산, SVM Classificaion, BoundingBoxRegression

## Fast R-CNN
- Region Proposal 추출, 전체 Image CNN 연산, RoI Projection 및 RoI Pooling, Classification, Bounding Box Regression

## RCNN Family Structure & Score
![RCNN_FamilyStructure](img/RCNN_FamilyStructure.png)

![RCNN_Score](img/RCNN_Score.png)

<hr>


## 문제점 & 연구배경 
- 기존 Region Proposal의 Selective Search 알고리즘이 CNN 외부에서 연산되므로 해당 ROI 생성단계가 병목임
- Region Proposal도 Selective search 없이 CNN - (Classification | bounding box regression) 안에서 해보자.

## 구조
- Faster R-CNN = RPN + Fast R-CNN 
![FasterRCNN3](img/FasterRCNN3.png)

![Faster_RCNN](img/Faster_RCNN.PNG)

<hr>

## Faster R-CNN 특징
- Region Proposal 단계를 CNN 과정에 넣음
- ROI Pooling을 사용하여 FC Layer의 Input size를 맞춰줌

- Faster R-CNN에서 Detection에 쓰인 Conv Feature를 RPN에서도 공유해서 RoI 생성역시 CNN Level에서 수행 및 속도 향상

![FasterRCNN2](img/FasterRCNN2.png)

<hr>

## RPN
- RPN (Region Proposal Network)
- RPN은 Image를 입력받아 사각형 형태의 Object Proposal과 Objectness를 뽑음
- RPN의 Input : 이전 CNN 모델에서 뽑아낸 Feature map
- Region Proposal을 생성하기 위해 Feature map위에 n x n window를 sliding 함
- Object의 크기와 비율이 어떻게 될지 모르므로 K개의 anchor box를 미리 정의
    - **1x1, 1x2, 2x1 ==> 비율 3종류(0.5, 1, 2)에 따라 => 총 9개** 
- CNN에서 뽑아낸 Feature Map에 대해 3x3 conv filter 256개 연산, (depth 256)
    - 그 후 1x1 conv 두개를 이용한 classification과 bbox regression을 계산
    - n x n x 4  &  1x1 conv 2개  ===> 4 x 2 의 FC 와 같음.
    
- RPN의 Bbox Regression = IoU기준으로 작동
    - IoU > 0.7 => 1 (객체 - Positive example)
    - IoU < 0.3 => -1 (Background 인것 - negative example)
    - 0.7 ~ 0.3 => 0 (애매한 객체, 학습데이터 이용x)
    - 최종적으로 IoU(가 0.7보다 크거나, 한 지점에서 모든 anchor Box 중 IoU가 가장 큰 Anchor Box를 Positive Example로 만든다.
- RPN의 Classification
    - binary classification (물체 존재 여부판단, Objectness)

## Anchor Box
- 동일한 크기의 sliding window를 이동
- window의 위치를 중심으로 사전에 정의된 다양한 비율/크기의 anchor box들을 적용하여 feature 추출
- image/feature pyramids처럼 image 크기를 조정할 필요 X
- multiple-scaled sliding window처럼 filter 크기를 변경할 필요 X
- 계산효율이 높은 방식

![FasterRCNN](img/FasterRCNN.png)

<hr>

## Loss Function for RPN
- i번째 Anchor에 대하여 Ground Truth(gt)와 Prediction 값 비교
    - RPN의 Class(물체가 있는지 없는지)
    - Box의 위치를 학습
- L_cls => Classification은 Log Loss를 사용,
- L_reg => Box Regression은 Smooth L1 Loss를 사용

![faster_rcnn_loss](img/faster_rcnn_loss.png)

<hr>

## 여러 Proposal에 대한 NMS
- NMS (Non-Maximu Suppression)
- RPN 모델을 통한 여러 개의 Proposal 결과
- NMS 알고리즘을 통한 Proposal의 갯수를 줄임

## NMS 순서
- (1) Box들의 score (confidence) 기준 정렬
- (2) score가 가장 높은 box부터 다른 Box들과의 IoU를 계산해서 0.7이상이면 같은 객체를 Detect한 box의 경우 라고 생각
- (3) 최종적으로 각 Object별로 Score가 가장 높은 box만 남음

![NMS전후](img/NMS전후.png)

## ROI Sampling
- 보통 Training 후 NMS를 거치면, 2000개의 ROI가 남음
- Positive:Negative 비율이 1:1이 되도록 RoI를 Sampling
- 256개를 Sampling시 128개의 Positive, 128개의 Negative 

<hr>

## Faster R-CNN 문제점
- 정확도는 높았으나 연산적인 측면에서 7FPS => 실시간적으로 활용 불가
- 해당과정이 2Stage로 나눠져 있는게 문제이다 -> yolo의 등장

## 참고문헌
- https://ganghee-lee.tistory.com/37
- https://nuggy875.tistory.com/33
- https://curt-park.github.io/2017-03-17/faster-rcnn/